In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
widthStripesAmount=48
heightStripesAmount=64


In [2]:
def CalculateStripMiddle(stripe, stripeHeigh):

    imgResized=cv2.resize(stripe, (int(stripeHeigh/4), 1),  interpolation = cv2.INTER_AREA)[0,:]
    #plt.plot(imgResized)
    #plt.show()
    maxims=[]
    for i in range (5, imgResized.shape[0]-6, 1):
        if(imgResized[i-5]<=imgResized[i] and imgResized[i]>imgResized[i+5]):
            if(imgResized[i-4]<=imgResized[i] and imgResized[i]>imgResized[i+4]):
                if(imgResized[i-3]<=imgResized[i] and imgResized[i]>imgResized[i+3]):
                    if(imgResized[i-2]<=imgResized[i] and imgResized[i]>imgResized[i+2]):
                        if(imgResized[i-1]<=imgResized[i] and imgResized[i]>imgResized[i+1]):
                            #print(i*4)
                            maxims.append(i*4)
    maxims=np.array(maxims)
    diffs=np.diff(maxims)
    ind=np.argpartition(diffs,-2)[-2:]
    if(ind[0]>ind[1]): ind[0]+=1
    else: ind[1]+=1
    StripMiddle=int(np.mean(maxims[ind]))
    return StripMiddle

def WidthMiddleFind(img, stripesAmount):
    height, width=img.shape
    stripesMeans=[]
    for i in range (0, height-int(height/stripesAmount), int(height/stripesAmount)):
        stripe=img[i: i+int(height/stripesAmount), : ]
        #print(CalculateStripMiddle(stripe))
        oneStripeMean=CalculateStripMiddle(stripe, width)
        stripesMeans.append(oneStripeMean)
    widthMiddle=int(np.round(np.mean(stripesMeans)))
    return widthMiddle

def WidthHeighFind(img, stripesAmount):
    height, width=img.shape
    stripesMeans=[]
    for i in range (0, width-int(width/stripesAmount), int(width/stripesAmount)):
        stripe=img[:, i: i+int(width/stripesAmount) ]
        stripe=stripe.transpose()
        #print(CalculateStripMiddle(stripe))
        oneStripeMean=CalculateStripMiddle(stripe, height)
        stripesMeans.append(oneStripeMean)
    heightMiddle=int(np.round(np.mean(stripesMeans)))
    return heightMiddle

def FindMiddle(img, widthStripesAmount, heightStripesAmount):
    img=img[4:484, 4:644]
    
    middleWidth=WidthMiddleFind(img, widthStripesAmount)
    middleHeight=WidthHeighFind(img, heightStripesAmount)
    return middleWidth, middleHeight

def CalculateEpsilon(img, widthStripesAmount, heightStripesAmount):
    middleWidth, middleHeight=FindMiddle(img, widthStripesAmount, heightStripesAmount)
    #print(middleWidth, middleHeight)
    #img = cv2.imread("../data/raw/1channel/medianedphoto/00000.png", cv2.IMREAD_GRAYSCALE)
    img=img[4:484, 4:644]
    stripeWidth=img.shape[1]
    stripe=img[(middleHeight-15):(middleHeight+15),:]

    #print(stripe.shape)
    imgResized=cv2.resize(stripe, (int(stripeWidth/4), 1),  interpolation = cv2.INTER_AREA)[0,:]
    #print(imgResized.shape)
    #plt.plot(imgResized)
    #plt.show()

    maxims=[]
    for i in range (5, imgResized.shape[0]-6, 1):
        if(imgResized[i-5]<=imgResized[i] and imgResized[i]>imgResized[i+5]):
            if(imgResized[i-4]<=imgResized[i] and imgResized[i]>imgResized[i+4]):
                if(imgResized[i-3]<=imgResized[i] and imgResized[i]>imgResized[i+3]):
                    if(imgResized[i-2]<=imgResized[i] and imgResized[i]>imgResized[i+2]):
                        if(imgResized[i-1]<=imgResized[i] and imgResized[i]>imgResized[i+1]):
                            maxims.append(i*4)

    maxims=np.array(maxims)
    diffs=np.diff(maxims)
    ind=np.argpartition(diffs,-2)[-2:]
    #print("maxims:", maxims)
    #print(ind)
    #print(maxims)
    #if(ind[0]>ind[1]): ind[0]+=1
    #else: ind[1]+=1
    StripMiddle=int(np.mean(maxims[ind]))
    #StripMiddle=300
    #print(StripMiddle)
    halfMaxims=[i for i in maxims if i>=StripMiddle ]
    #print(halfMaxims)
    halfMaxims=[(i-StripMiddle)*(i-StripMiddle) for i in halfMaxims]
    halfMaxims.insert(0,0)
    #print(halfMaxims)
    pointsNumber=len(halfMaxims)
    
    #PlotParameters


    #a=(halfMaxims[pointsNumber-1]-halfMaxims[1])/(pointsNumber-1)
    #b=halfMaxims[pointsNumber-1]-pointsNumber*a
    #a=(halfMaxims[3]-halfMaxims[1])/(3-1)
    #b=halfMaxims[1]-a
    a=(halfMaxims[2]-halfMaxims[1])/1
    b=halfMaxims[1]-a

    #print(a)
    #print(b)
    epsilon=1-abs(b/a)
    #print(epsilon)
    #plt.plot(halfMaxims)
    #plt.show()
    return epsilon

    

In [3]:
img = cv2.imread("../data/raw/1channel/medianedphoto/00000.png", cv2.IMREAD_GRAYSCALE)

print(CalculateEpsilon(img, widthStripesAmount, heightStripesAmount))


0.34505444646098005
